## Functions and Chat Completion
Jeremy Howard provided an overview of using ChatGPT to create you own "Code Interpreter" that can be used by LLMs to call python code.

See:
https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb

In [1]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [27]:
from fastcore.utils import nested_idx
def response(compl): print(nested_idx(compl, 'choices', 0, 'message', 'content'))

In [2]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [3]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [4]:
schema(sums)

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [5]:
from openai import ChatCompletion,Completion

In [6]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [7]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [8]:
m = c.choices[0].message
m

<OpenAIObject at 0x11179ee50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

In [9]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [10]:
funcs_ok = {'sums', 'python'}

In [11]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [12]:
call_func(c)

9

In [13]:
import ast

In [14]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [15]:
run("""
a=1
b=2
a+b
""")

3

In [16]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [32]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [33]:
call_func(c)

KeyboardInterrupt: Interrupted by user

In [23]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[schema(python)],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "python", "content": "479001600"}])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [28]:
response(c)

12 factorial, denoted as 12!, is equal to 479,001,600.


In [29]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

/var/folders/c5/gc7vgjds6tq1jy3b143hjtnm0000gn/T/ipykernel_42954/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [30]:
response(c)


The capital of France is Paris.
